In [50]:
import re
import pandas as pd

In [51]:
# filename = "twenty_newsgroups_calib_0_passive_learning_eval_full_std_xent_only.xlsx"
filename = "tiny_imagenet_CLIP_calib_0_passive_learning_eval_full_std_xent_only.xlsx"

# write a regular expression to extract the method name from the filename (e.g. 'twenty_newsgroups_calib_0_passive_learning_eval_full_std_xent_only.xlsx')
search = re.search(r"(.*)_calib_0_(.*)_eval_full_(.*).xlsx", filename)
dataset, method, training_time_method = (
    search.group(1),
    search.group(2),
    search.group(3),
)

training_time_method = "vanilla" if training_time_method == "std_xent_only" else training_time_method

src = pd.read_excel(f"outputs/{filename}", index_col=0)
src.calib_conf.fillna("None", inplace=True)

caption = f"Auto-labeling error and coverage of post-hoc methods for {training_time_method} training-time methods via {method.replace('_', ' ')} on {dataset.replace('_', ' ')} dataset."
label = f"table:{dataset}_{method}_{training_time_method}"

In [52]:
column_alignments = ['l', 'l', 'l', 'l']
excel_to_text_header_mapping = {
    'calib_conf': 'Post-hoc Method',
    'training_conf': 'Training-time Method',
    'Auto-Labeling-Err': 'Auto-labeling Error',
    'Coverage': 'Coverage',
}
conf_to_text_mapping = {
    'None': ' - ',
    'auto_label_opt_v0': 'Ours',
    'dirichlet': 'Dirichlet',
    'histogram_binning_top_label': 'Histogram binning top label',
    'scaling_binning': 'Scaling binning',
    'temp_scaling': 'Temperature scaling',
    'std_cross_entropy': 'Vanilla',
}

In [53]:
def generate_theader(values):
    return ' & '.join(map(lambda x: x.ljust(30), values)) + ' \\\\'

In [54]:
def generate_tbody(df):

    coverage_max_id = src['Coverage-Mean'].argmax()
    auto_labeling_err_min_id = src['Auto-Labeling-Err-Mean'].argmin()

    table_rows = []
    for idx, row in df.iterrows():
        table_row = []
        for col in excel_to_text_header_mapping.keys():
            if col == "Auto-Labeling-Err":
                mean = row[col + "-Mean"]
                sd = row[col + "-Std"]
                str_val = rf"{mean:.3f} $\pm$ {sd:.3f}"
                if idx == auto_labeling_err_min_id:
                    str_val = rf"\textbf{{{str_val}}}"
                table_row.append(str_val)
            elif col == "Coverage":
                mean = row[col + "-Mean"]
                sd = row[col + "-Std"]
                str_val = rf"{mean:.3f} $\pm$ {sd:.3f}"
                if idx == coverage_max_id:
                    str_val = rf"\textbf{{{str_val}}}"
                table_row.append(str_val)
            else:
                table_row.append(f"{conf_to_text_mapping[row[col]]}")
        table_row = map(lambda x: x.ljust(30), table_row)
        table_rows.append(' & '.join(table_row))
    return ' \\\\\n        '.join(table_rows) + ' \\\\'

In [55]:
prefix = rf"""\begin{{table*}}[t]
    \centering
    \begin{{tabular}}{'{' + ''.join(column_alignments) + '}'}
        \toprule
        {generate_theader(excel_to_text_header_mapping.values())}
        \midrule
        {generate_tbody(src)}
        \bottomrule
    \end{{tabular}}
    \caption{'{' + caption + '}'}
    \label{'{' + label + '}'}
\end{{table*}}
"""

In [56]:
print(prefix)

\begin{table*}[t]
    \centering
    \begin{tabular}{llll}
        \toprule
        Post-hoc Method                & Training-time Method           & Auto-labeling Error            & Coverage                       \\
        \midrule
         -                             & Vanilla                        & 6.505 $\pm$ 0.417              & 20.521 $\pm$ 1.279             \\
        Ours                           & Vanilla                        & \textbf{2.154 $\pm$ 3.127}     & \textbf{38.189 $\pm$ 2.369}    \\
        Dirichlet                      & Vanilla                        & 12.275 $\pm$ 1.279             & 11.903 $\pm$ 0.695             \\
        Histogram binning top label    & Vanilla                        & 10.659 $\pm$ 0.622             & 13.634 $\pm$ 0.437             \\
        Scaling binning                & Vanilla                        & 10.326 $\pm$ 0.209             & 12.879 $\pm$ 1.126             \\
        Temperature scaling            & Vanilla             